# Аннотация типов 

Динамическая типизация с одной стороны делает язык `python` чрезвычайно гибким, а с другой стороны и что-то отнимает. 

## Преимущества статической типизации

### Обнаружение ошибок  

Во-первых, статическая типизация позволяет обнаруживать ошибки на этапе компиляции. 

В качестве примера рассмотрим следующий код на `C++`.

```C++
#include<iostream>
#include<string>

int f(int x){
    return x + 1;
}

int main(){
	std::string s = "abc";
	std::cout << negation(s);
}
```

Его компиляция средствами `g++` приводит к ошибке компиляции со следующим сообщением.

```
error: cannot convert 'std::__cxx11::string {aka std::__cxx11::basic_string<char>}' to 'int' for argument '1' to 'int f(int)'
  std::cout << f(s);
                  ^
```

Эта ошибка связана с тем, что функция `f` объявлена с целочисленным параметром `x`, а вызывается она со строковым значением. 

Сравним это с аналогичным кодом на `python`. 
```python
def f(x):
    return x + 1

s = "abc"
print(f(s))
```

Запуск этого кода приведет к возникновению следующей ошибки. 
```
Traceback (most recent call last):
  File ".\static_typing.py", line 5, in <module>
    print(f(s))
  File ".\static_typing.py", line 2, in f
    return x + 1
TypeError: can only concatenate str (not "int") to str
```

Возникает ошибка при попытке сложить строку и число. Существенная разница заключается в том, что эта ошибка возникает во время исполнения программы, а не на предварительно этапе. 

Вообще говоря в `python` существуют [инструменты статического анализа кода](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%BA%D0%BE%D0%B4%D0%B0), которые пытаются обнаружить ошибки в программе, анализируя её исходный код, но не запуская её. [Mypy](http://mypy-lang.org/) --- один из таких инструментов.

Применение его к тому же исходному коду не приведет к обнаружению ошибок. 

Однако если добавить подсказку типа для параметра `x` следующим образом. 
```python
def f(x: int):
    return x + 1

s = "abc"
print(f(s))
```
Тогда `mypy` обнаружит несоответствие типа параметра `x` и переменной `s`.

### Документирование кода

Рассмотрим функцию `integrate` на языке `С++` с следующей сигнатурой. 

```c++
double integrate(std::function<double(double)> f, double a, double b)
```

Исходя из типов параметров, легко предположить, что
1) в качестве первого параметра необходимо передать функцию $f$, которая должна принимать на вход число типа `double` и возвращать тоже число типа `double`;  
2) в качестве второго и третьего параметров ожидаются границы отрезка интегрирования в виде чисел типа `double`;
3) функция `integrate` возвращает результат интегрирования в виде числа типа `double`.

Т.е. сигнатура функции подсказывает пользователю, как её правильно вызывать, тем самым осуществляя документирующую функцию. А тем рассмотрим, как будет выглядеть сигнатура аналогичной функции в языке с динамической типизацией. 

```python
def integrate(f, a, b):
    ...
```

Смотря только на сигнатуру можно только предположить, что `f` --- функция, а параметры `a` и `b` --- границы интегрирования. Но только из сигнатуры не удается понять, какое поведение ожидается от функции `f`, а также в каком виде возвращается результат вычисления интеграла: число или может быть структура данных с дополнительной отладочной информацией?

Схожего документирующего эффекта можно добиться следующей аннотацией функции `integrate`, которая является абсолютно корректным кодом с точки зрения интерпретатора `python`.

```python
from collections.abc import Callable

def integrate(f: Callable[[float], float], a: float, b: float) -> float:
    ...
```

### Автозаполнение в IDE

Среды разработки стараются упростить процесс разработки и опытные программисты этим пользуются. В частности `IDE` на основе типов переменных и параметров функции могут определить список доступных методов и предложить возможные варианты для автоматического дополнения. 

```{figure} /_static/lecture_specific/dynamic_typing/cpp_hint.gif
```

В анимации выше среда разработки с каждой набранной буквой метода `push_back` подсказывает все более узкий список возможных продолжений, т.к. среде разработки известно, что переменная `x` является контейнером [std::vector](https://en.cppreference.com/w/cpp/container/vector), все методы которого её тоже известны. 

В случае с динамической типизацией вместо получается совершенно другая картина.
```{figure} /_static/lecture_specific/dynamic_typing/python_no_hint.gif
```
При вызове функции `f` на месте переменной `x` может оказаться объект любого типа. Даже если программисту известно, что это всегда будет список, у которого есть метод `append`, то среде разработке это отнюдь не очевидно. Это приводит к тому, что она не может подсказать возможное продолжение, т.к. список возможных продолжений близок к безграничному. 

Однако если подсказать среде разработке, что `x` --- переменная типа `list`, то она начнет подсказывать. 

```{figure} /_static/lecture_specific/dynamic_typing/python_hint.gif
```